# About the notebook

Do exploratory data analysis and sentiment analysis 

# About the data

Dataset consists of tweets of Trump.


# About this Notebook

- <a href =#1> 1. Import packages and libraries  </a>
- <a href =#2> 2. Read the data </a>
- <a href =#3> 3. Handle null values </a>
- <a href =#4> 4. Exploratory data analysis </a>
- <a href =#5> 5. Clean the data </a>
- <a href =#6> 6. Apply data cleaning steps to data</a>
- <a href =#7> 7. Visualize the most Common words in the tweets </a>
- <a href =#8> 8.  Get the sentiment analysis of the tweets </a>
- <a href =#9> 9.  Save the data </a>

<br>
Credit goes to:
https://www.kaggle.com/shailaja4247/sentiment-analysis-of-tweets-wordclouds-textblob#About-the-data

<div id=1> <h2>  1. Import packages and libraries  </h2> </div> 

In [3]:
! pip install plotly

  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11435 sha256=538a8e6f4a108b55a199e5b1bfc4c7ee91c98f3d3fcc189ae5b6da9389d85a62
  Stored in directory: c:\users\54546\appdata\local\pip\cache\wheels\c4\a7\48\0a434133f6d56e878ca511c0e6c38326907c0792f67b476e56
Successfully built retrying
  Using cached plotly-4.14.1-py2.py3-none-any.whl (13.2 MB)


In [24]:
! pip install emoji

  Created wheel for emoji: filename=emoji-0.6.0-py3-none-any.whl size=49720 sha256=4bbaaea2bb28ac8824a0faa680544a5ab5b57ee8bc556afae1d05aa38f63baef
  Stored in directory: c:\users\54546\appdata\local\pip\cache\wheels\43\3d\82\e7baffa5e86346c6178d7750dba6e8ef063282a37fc563f8f8
Successfully built emoji


In [44]:
! pip install textblob

In [102]:
import re
import string
import numpy as np 
import random
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('punkit')

from tqdm import tqdm
import os
import nltk
import random

import warnings
warnings.filterwarnings("ignore")

import os

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\54546\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\54546\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\54546\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Error loading punkit: Package 'punkit' not found in index


**Below is a helper Function which generates random colors which can be used to give different colors to your plots.Feel free to use it**

<div id=2> <h2> 2. Read the data  </h2></div> 

In [103]:
import pandas as pd
import numpy as np
train=pd.read_csv('../data/input/realdonaldtrump_20170120-20191231.csv')

In [104]:
print(train.shape)

(9690, 8)


In [105]:
train.describe()

,id,retweets,favorites
count,9.690000e+03,9690.000000,9690.000000
mean,1.055627e+18,17968.644066,79928.356347
std,1.103727e+17,10248.538428,39531.033392
min,8.224210e+17,535.000000,2317.000000
25%,9.675642e+17,12113.000000,55761.500000
50%,1.069730e+18,16155.500000,72963.500000
75%,1.155308e+18,21364.000000,95168.250000
max,1.212210e+18,302269.000000,835575.000000


In [106]:
train.isna().sum()

id              0
link            0
content         0
date            0
retweets        0
favorites       0
mentions     7850
hashtags     8911
dtype: int64

hashtags have lot of null values, mentions also have lot of null values.

<div id=3> <h2> 3. Handle null values </h2> </div>

In [107]:
train['hashtags'].unique()[:10]

array([nan, '#InaugurationDay', '#AmericaFirst', '#InaugurationDay,#MAGA',
       '#MAGA', '#MarchForLife', '#NeverForgetpic', '#SuperBowl', '#1',
       '#AmericaFirstpic'], dtype=object)

In [108]:
train['mentions'].unique()[:10]

array([nan, '@FoxNews', '@WhiteHouse', '@FoxNews,@CNN', '@DavidMuir,@ABC',
       '@DHSgov', '@romoabcnews,@DavidMuir,@POTUS,@ABCWorldNews,@ABC2020',
       '@ABC', '@GOP,@VP,@SenateMajLdr,@SpeakerRyan',
       '@SeanHannity,@FoxNews'], dtype=object)

1. hashtags: This column does not have something very significant for us to analyse. We have mostly words like #ixzz4 etc which make no sense and we wont be able to analyse their sentiment. So we can get rid of this column
2. mentions usually mention another person and we wont really get any sentiment by analysing that. mentions too does not have anything siginificant for us to analyse. Hence we can do away with that too. Let us drop columns which dont have any siginificantly useful information. 

In [109]:
train=train.drop(['link','mentions','hashtags'], axis=1)

<div id=4> <h2> 4. EDA (Exploratory data analysis)  </h2></div>

In [110]:
train.head()

,id,content,date,retweets,favorites
0,8.224210e+17,It all begins today! I will see you at 11:00 A...,1/20/2017 6:31,59617,238180
1,8.225020e+17,Today we are not merely transferring power fro...,1/20/2017 11:51,17022,97230
2,8.225020e+17,"power from Washington, D.C. and giving it back...",1/20/2017 11:51,15441,80281
3,8.225020e+17,What truly matters is not which party controls...,1/20/2017 11:52,36987,161112
4,8.225020e+17,"January 20th 2017, will be remembered as the d...",1/20/2017 11:53,56418,201434


Lets look at the distribution of reviews in the train set

<div id=5> <h2>5. Data Preprocessing </h2> </div>

Let's first clean the data, remove stopwords etc and perform basic pre-processing

In [111]:
train["content"] = train["content"].str.lower()
china_related_list = ['chinese', 'huawei', 'xi jinping', 'beijing', 'trade', 'tariff','tax']
train_china_related = train.set_index('content').filter(like='china', axis=0)
for word in china_related_list:
    train_china_related = pd.concat([train_china_related, train.set_index('content').filter(like=word, axis=0)]).drop_duplicates()
train_china_related.reset_index(inplace=True, drop=False)

In [112]:
train_china_related[['Date','Time']] = train_china_related.date.str.split(" ",expand = True,)
col_list = train_china_related.columns
train_china_related = train_china_related[[col_list[1], col_list[0], col_list[2],col_list[5], col_list[6], col_list[3], col_list[4]]]

In [113]:
train_china_related

,id,content,date,Date,Time,retweets,favorites
0,8.300480e+17,the failing @ nytimes does major fake news chi...,2/10/2017 7:35,2/10/2017,7:35,19622,92478
1,8.427240e+17,north korea is behaving very badly. they have ...,3/17/2017 8:07,3/17/2017,8:07,17371,76787
2,8.475730e+17,the meeting next week with china will be a ver...,3/30/2017 17:16,3/30/2017,17:16,10038,49913
3,8.507230e+17,it was a great honor to have president xi jinp...,4/8/2017 9:50,4/8/2017,9:50,8768,58575
4,8.517670e+17,i explained to the president of china that a t...,4/11/2017 6:59,4/11/2017,6:59,15866,70405
...,...,...,...,...,...,...,...
917,1.197500e+18,"bob mueller, after spending two years and 45 m...",11/21/2019 6:58,11/21/2019,6:58,16615,64749
918,1.205480e+18,how do you get impeached when you have done no...,12/13/2019 7:42,12/13/2019,7:42,29245,133255
919,1.207290e+18,"....said, “i’m going to clean up washington, i...",12/18/2019 7:18,12/18/2019,7:18,7339,30729
920,1.208030e+18,i will be signing our 738 billion dollar defen...,12/20/2019 8:19,12/20/2019,8:19,32864,167191


In [114]:
train_china_related['Date'] = pd.to_datetime(train_china_related['Date'])
train_china_related = train_china_related[train_china_related['Date']> pd.to_datetime('1/1/2018')]

In [115]:
train = train_china_related

In [116]:
train

,id,content,date,Date,Time,retweets,favorites
43,9.616720e+17,i will be meeting with henry kissinger at 1:45...,2/8/2018 12:44,2018-02-08,12:44,11239,70357
44,9.652030e+17,i never said russia did not meddle in the elec...,2/18/2018 6:33,2018-02-18,6:33,26495,111426
45,9.714030e+17,china has been asked to develop a plan for the...,3/7/2018 9:10,2018-03-07,9:10,15544,75828
46,9.725060e+17,chinese president xi jinping and i spoke at le...,3/10/2018 10:15,2018-03-10,10:15,15895,80544
47,9.789390e+17,received message last night from xi jinping of...,3/28/2018 5:16,2018-03-28,5:16,19918,90508
...,...,...,...,...,...,...,...
917,1.197500e+18,"bob mueller, after spending two years and 45 m...",11/21/2019 6:58,2019-11-21,6:58,16615,64749
918,1.205480e+18,how do you get impeached when you have done no...,12/13/2019 7:42,2019-12-13,7:42,29245,133255
919,1.207290e+18,"....said, “i’m going to clean up washington, i...",12/18/2019 7:18,2019-12-18,7:18,7339,30729
920,1.208030e+18,i will be signing our 738 billion dollar defen...,12/20/2019 8:19,2019-12-20,8:19,32864,167191


**Removing weird spaces**

In [117]:
def remove_spaces(text):
    text=text.strip()
    text=text.split()
    return ' '.join(text)

<b> Spelling Correction </b>

In [118]:
def edits1(word):
    letters='abcdefghijklmnopqrstuvwxyz'
    splits=[(word[:i], word[i:]) for i in range(len(word)+1)]
    deletes=[L+R[1:] for L,R in splits if R]
    transposes=[L+R[1] +R[0] + R[2:] for L,R in splits if len(R)>1]
    replaces = [L+c+R[1:] for L,R in splits if R for c in letters]
    inserts = [L+c+ R for L,R in splits for c in letters]
    return set(deletes+transposes+replaces+inserts)
def edits2(word):
    return(e2 for e1 in edits1(word) for e2 in edits1(e1))

<b> Contraction </b>

In [119]:
contraction = {'cause':'because',
              'aint': 'am not',
              'aren\'t': 'are not'}

def mapping_replacer(x,dic):
    for words in dic.keys():
        if ' ' + words + ' ' in x:
            x=x.replace(' '+ words +' ' ,' '+dic[words]+' ' )
    return x

<b> Stemming, lemmetisation and tokenisation
</b>

In [120]:
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.lancaster import LancasterStemmer

nltk.LancasterStemmer
ls = LancasterStemmer()
lem = WordNetLemmatizer()
def lexicon_normalization(text):
    words = word_tokenize(text) 
    
    
    # 1- Stemming
    words_stem = [ls.stem(w) for w in words]
    
    # 2- Lemmatization
    words_lem = [lem.lemmatize(w) for w in words_stem]
    return words_lem

<b> Handling emojis </b>

In [121]:
import emoji
import re 
#from emot.emo_unicode import UNICODE_EMO
def convert_emojis(text):
    for emot in emoji.UNICODE_EMOJI:
        text = re.sub(r'('+emot+')', "_".join(emoji.UNICODE_EMOJI[emot].replace(",","").replace(":","").split()), text)
    return text

<b>Removing links, brackets, numbers, punctuations etc. </b>


In [122]:
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('\'','', text)
    
    return text

<b> Handling stopwords </b>

In [123]:
from collections import Counter
def remove_stopword(text):
    stop_words = stopwords.words('english')
    stopwords_dict = Counter(stop_words)
    text = ' '.join([word for word in text.split() if word not in stopwords_dict])
    return text

<b> Tokenisation </b>

In [124]:
def tokenise(text):
    words = word_tokenize(text) 
    return words


<div id = 6> <h2> 6. Apply data cleaning steps to data </h2> </div>

Cleaning Regex Expressions from data

In [125]:
import re
train['content'] = train['content'].map(lambda x: re.sub(r'\W+', ' ', x))
train['content'] = train['content'].replace(r'\W+', ' ', regex=True)


In [126]:
train.head()

,id,content,date,Date,Time,retweets,favorites
43,9.616720e+17,i will be meeting with henry kissinger at 1 45...,2/8/2018 12:44,2018-02-08,12:44,11239,70357
44,9.652030e+17,i never said russia did not meddle in the elec...,2/18/2018 6:33,2018-02-18,6:33,26495,111426
45,9.714030e+17,china has been asked to develop a plan for the...,3/7/2018 9:10,2018-03-07,9:10,15544,75828
46,9.725060e+17,chinese president xi jinping and i spoke at le...,3/10/2018 10:15,2018-03-10,10:15,15895,80544
47,9.789390e+17,received message last night from xi jinping of...,3/28/2018 5:16,2018-03-28,5:16,19918,90508


In [127]:
train['content']=train['content'].apply(lambda x: mapping_replacer(x, contraction))

In [128]:
train['content'] = train['content'].apply(lambda x:clean_text(x))

In [129]:
train['content']=train['content'].apply(lambda x: remove_stopword(x))


In [130]:
train['content']=train['content'].apply(lambda x: lexicon_normalization(x))

In [131]:
train.head()

,id,content,date,Date,Time,retweets,favorites
43,9.616720e+17,"[meet, henry, kiss, discus, nor, kore, chin, m...",2/8/2018 12:44,2018-02-08,12:44,11239,70357
44,9.652030e+17,"[nev, said, rus, meddl, elect, said, may, rus,...",2/18/2018 6:33,2018-02-18,6:33,26495,111426
45,9.714030e+17,"[chin, ask, develop, plan, year, on, bil, doll...",3/7/2018 9:10,2018-03-07,9:10,15544,75828
46,9.725060e+17,"[chine, presid, xi, jinp, spok, leng, meet, ki...",3/10/2018 10:15,2018-03-10,10:15,15895,80544
47,9.789390e+17,"[receiv, mess, last, night, xi, jinp, chin, me...",3/28/2018 5:16,2018-03-28,5:16,19918,90508


<div id=7><h2>7. Visualize the most Common words in the tweets </h2></div>

In [132]:
top = Counter([item for sublist in train['content'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Blues')

,Common_words,count
0,chin,358
1,u,328
2,trad,308
3,tax,226
4,gre,221
5,tariff,206
6,deal,189
7,country,176
8,stat,132
9,year,127


In [133]:
blacklist = ['http','https','www','com', 'ev','u','ly','pic','would']

def remove_words(text):
    text = [i for i in text if (i not in blacklist)]
    return text

In [134]:
#train['content']=remove_words(train['content'])

train['content'] = train['content'].apply(lambda x: [i for i in x if i not in blacklist])

In [135]:
fig = px.bar(temp, x="count", y="Common_words", title='Commmon Words in Selected Text', orientation='h', 
             width=700, height=700,color='Common_words')
fig.show()

In [136]:
top = Counter([item for sublist in train['content'] for item in sublist])
temp = pd.DataFrame(top.most_common(20))
temp = temp.iloc[1:,:]
temp.columns = ['Common_words','count']
temp.style.background_gradient(cmap='Purples')

,Common_words,count
1,trad,308
2,tax,226
3,gre,221
4,tariff,206
5,deal,189
6,country,176
7,stat,132
8,year,127
9,bil,125
10,presid,121


<div id = 8><h2>8. Get the sentiment analysis of the tweets</h2></div>

Let's look at the most common words in different sentiments

In [137]:
from textblob import TextBlob

def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(tweet) 
    
    # set sentiment 
    if analysis.sentiment.polarity > 0:
        return 1
    elif analysis.sentiment.polarity == 0: 
        return 0
    else: 
        return -1
    # return analysis.sentiment.polarity

In [138]:
train['sentiment']=train['content'].apply(lambda x: get_tweet_sentiment(' '.join(x)))

In [139]:
train.head()

,id,content,date,Date,Time,retweets,favorites,sentiment
43,9.616720e+17,"[meet, henry, kiss, discus, nor, kore, chin, m...",2/8/2018 12:44,2018-02-08,12:44,11239,70357,0
44,9.652030e+17,"[nev, said, rus, meddl, elect, said, may, rus,...",2/18/2018 6:33,2018-02-18,6:33,26495,111426,1
45,9.714030e+17,"[chin, ask, develop, plan, year, on, bil, doll...",3/7/2018 9:10,2018-03-07,9:10,15544,75828,1
46,9.725060e+17,"[chine, presid, xi, jinp, spok, leng, meet, ki...",3/10/2018 10:15,2018-03-10,10:15,15895,80544,0
47,9.789390e+17,"[receiv, mess, last, night, xi, jinp, chin, me...",3/28/2018 5:16,2018-03-28,5:16,19918,90508,-1


In [140]:
Positive_sent = train[train['sentiment']==1]
Negative_sent = train[train['sentiment']==0]
Neutral_sent = train[train['sentiment']==-1]

<b> Overall tweet sentiment </b>

In [141]:
print('Number of tweets with positive sentiment', Positive_sent['sentiment'].count())
print('Number of tweets with negative sentiment', Negative_sent['sentiment'].count())
print('Number of tweets with neutral sentiment', Neutral_sent['sentiment'].count())


Number of tweets with positive sentiment 412
Number of tweets with negative sentiment 105
Number of tweets with neutral sentiment 182


<b><i>So we see that overall the tweets are neutral in nature, followed by positive sentiment for the time in which they are analysed.</i></b>

<div id = 9><h2>9. Save the data </h2></div>

In [142]:
train.to_csv('../output/results/textblob_prediction_data.csv')